In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql import functions as F
from pyspark.sql import Window

# https://repo1.maven.org/maven2/org/apache/iceberg/iceberg-spark-runtime-3.3_2.12/
# https://iceberg.apache.org/multi-engine-support/#apache-spark
# https://iceberg.apache.org/multi-engine-support/#apache-hive
import os

os.environ['AWS_ACCESS_KEY_ID']='andreyolv'
os.environ['AWS_REGION']='us-east1'
#os.environ['AWS_REGION']='us-east1'

spark = (SparkSession
    .builder
    .appName("etl-device-subscription")
    .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:3.3.1,"
                                    "com.amazonaws:aws-java-sdk-bundle:1.12.709,"
                                    "org.apache.iceberg:iceberg-spark-runtime-3.2_2.12:1.2.1,"
                                    "org.apache.iceberg:iceberg-hive-runtime:0.13.0,"
                                    "software.amazon.awssdk:bundle:2.25.39,"
                                    "software.amazon.awssdk:url-connection-client:2.25.39"
            )
    # S3 / Minio
    .config("spark.hadoop.fs.s3a.access.key", "andreyolv")
    .config("spark.hadoop.fs.s3a.secret.key", "andreyolv")
    .config("spark.hadoop.fs.s3a.endpoint", "http://minio.minio:9000")
    .config("spark.hadoop.fs.s3a.path.style.access", True)
    .config("spark.hadoop.fs.s3a.fast.upload", True)
    .config("spark.hadoop.fs.s3a.multipart.size", 104857600)
    .config("fs.s3a.connection.maximum", 100)
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
    .config("spark.hadoop.fs.s3a.aws.credentials.provider", "org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider")
    # Iceberg
    .config("spark.sql.extensions", "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions")
    .config("spark.sql.catalog.owshq", "org.apache.iceberg.spark.SparkCatalog")
    .config("spark.sql.catalog.owshq.s3.endpoint", "http://minio.minio:9000")
    # Catalog on Hive Metastore
    .config("spark.serializer", "org.apache.spark.serializer.KryoSerializer")
    .config("spark.sql.catalogImplementation", "hive")
    .config("spark.sql.catalog.owshq.type", "hive")
    .config("spark.hive.metastore.uris", "thrift://metastore.iceberg:9083")
    .config("spark.sql.hive.thriftServer.singleSession", "false")
    .config("spark.hive.metastore.schema.verification", "false")
    .enableHiveSupport()
    .getOrCreate()
)

In [3]:
spark.sql("""
CREATE SCHEMA iceberg_db LOCATION 's3a://lakehouse/iceberg/';
""")

DataFrame[]

In [5]:
spark.sql("""
SHOW SCHEMAS;
""").show()

+----------+
| namespace|
+----------+
|        db|
|       db2|
|       db3|
|       db4|
|   default|
|  delta_db|
|iceberg_db|
+----------+



In [6]:
spark.sql("SHOW DATABASEs;").show()

+----------+
| namespace|
+----------+
|        db|
|       db2|
|       db3|
|       db4|
|   default|
|  delta_db|
|iceberg_db|
+----------+



In [12]:
spark.sql("""
CREATE TABLE IF NOT EXISTS owshq.iceberg_db.table2 (
revenue int,
department string,
boss string)
USING ICEBERG
location 's3a://lakehouse/iceberg/table2'
""")

DataFrame[]

In [13]:
spark.sql("""
INSERT INTO owshq.iceberg_db.table2 (revenue, department, boss)
VALUES (10000, 'Sales', 'John Smith');
""")

DataFrame[]

In [14]:
spark.sql("""
SELECT * FROM owshq.iceberg_db.table2;
""").show()

+-------+----------+----------+
|revenue|department|      boss|
+-------+----------+----------+
|  10000|     Sales|John Smith|
+-------+----------+----------+



In [11]:
spark.sql("CREATE DATABASE db3;")

++
||
++
++



In [15]:
spark.sql("CREATE NAMESPACE owshq.db4;")

DataFrame[]

In [2]:
data = [
[10,'Direct Sales','Mario'],
[12,'Direct Sales','Joe'],
[20,'Online Sales','Sally'],
[25,'Online Sales','Dawn'],
]
 
df = spark.createDataFrame(data , ['revenue','department','boss'])
 
df.show()

+-------+------------+-----+
|revenue|  department| boss|
+-------+------------+-----+
|     10|Direct Sales|Mario|
|     12|Direct Sales|  Joe|
|     20|Online Sales|Sally|
|     25|Online Sales| Dawn|
+-------+------------+-----+



In [3]:
spark.sql("""CREATE TABLE IF NOT EXISTS owshq.db4.table1 (
revenue int,
department string,
boss string)
USING iceberg
location 's3://lakehouse/iceberg/table1'""")

Py4JJavaError: An error occurred while calling o65.sql.
: software.amazon.awssdk.core.exception.SdkClientException: Unable to load credentials from any of the providers in the chain AwsCredentialsProviderChain(credentialsProviders=[SystemPropertyCredentialsProvider(), EnvironmentVariableCredentialsProvider(), WebIdentityTokenCredentialsProvider(), ProfileCredentialsProvider(profileName=default, profileFile=ProfileFile(sections=[])), ContainerCredentialsProvider(), InstanceProfileCredentialsProvider()]) : [SystemPropertyCredentialsProvider(): Unable to load credentials from system settings. Access key must be specified either via environment variable (AWS_ACCESS_KEY_ID) or system property (aws.accessKeyId)., EnvironmentVariableCredentialsProvider(): Unable to load credentials from system settings. Secret key must be specified either via environment variable (AWS_SECRET_ACCESS_KEY) or system property (aws.secretAccessKey)., WebIdentityTokenCredentialsProvider(): Either the environment va

In [4]:
spark.sql("CREATE TABLE owshq.db4.new_table (col1 INT, col2 STRING);")

Py4JJavaError: An error occurred while calling o65.sql.
: org.apache.iceberg.exceptions.ValidationException: Invalid S3 URI, cannot determine scheme: file:/home/jovyan/spark-warehouse/db4.db/new_table/metadata/00000-5e1aaea8-a4a3-4e78-ac31-5d4cfb69db1a.metadata.json
	at org.apache.iceberg.exceptions.ValidationException.check(ValidationException.java:46)
	at org.apache.iceberg.aws.s3.S3URI.<init>(S3URI.java:57)
	at org.apache.iceberg.aws.s3.S3OutputFile.fromLocation(S3OutputFile.java:37)
	at org.apache.iceberg.aws.s3.S3FileIO.newOutputFile(S3FileIO.java:85)
	at org.apache.iceberg.BaseMetastoreTableOperations.writeNewMetadata(BaseMetastoreTableOperations.java:150)
	at org.apache.iceberg.hive.HiveTableOperations.doCommit(HiveTableOperations.java:214)
	at org.apache.iceberg.BaseMetastoreTableOperations.commit(BaseMetastoreTableOperations.java:127)
	at org.apache.iceberg.BaseMetastoreCatalog$BaseMetastoreCatalogTableBuilder.create(BaseMetastoreCatalog.java:165)
	at org.apache.iceberg.Cachin

In [13]:
spark.sql("CREATE SCHEMA iceberg_db LOCATION 's3://lakehouse/teste';")

Py4JJavaError: An error occurred while calling o63.sql.
: java.lang.RuntimeException: Failed to create namespace iceberg_db in Hive Metastore
	at org.apache.iceberg.hive.HiveCatalog.createNamespace(HiveCatalog.java:270)
	at org.apache.iceberg.spark.SparkCatalog.createNamespace(SparkCatalog.java:340)
	at org.apache.spark.sql.execution.datasources.v2.CreateNamespaceExec.run(CreateNamespaceExec.scala:47)
	at org.apache.spark.sql.execution.datasources.v2.V2CommandExec.result$lzycompute(V2CommandExec.scala:43)
	at org.apache.spark.sql.execution.datasources.v2.V2CommandExec.result(V2CommandExec.scala:43)
	at org.apache.spark.sql.execution.datasources.v2.V2CommandExec.executeCollect(V2CommandExec.scala:49)
	at org.apache.spark.sql.execution.QueryExecution$$anonfun$eagerlyExecuteCommands$1.$anonfun$applyOrElse$1(QueryExecution.scala:98)
	at org.apache.spark.sql.execution.SQLExecution$.$anonfun$withNewExecutionId$6(SQLExecution.scala:109)
	at org.apache.spark.sql.execution.SQLExecution$.withSQL

In [6]:
spark.sql(
"""
CREATE TABLE db.taxis
(
  trip_id bigint,
  trip_distance float,
  fare_amount double,
  store_and_fwd_flag string
)
PARTITIONED BY (vendor_id bigint) STORED BY ICEBERG;
""")

ParseException: 
Syntax error at or near 'ICEBERG'(line 9, pos 44)

== SQL ==

CREATE TABLE db.taxis
(
  trip_id bigint,
  trip_distance float,
  fare_amount double,
  store_and_fwd_flag string
)
PARTITIONED BY (vendor_id bigint) STORED BY ICEBERG;
--------------------------------------------^^^
